In [5]:
import glob
import datetime
import pandas as pd
import numpy as np


class PreProcess:
    def __init__(self, input, target, time, target_all, fill_cnt=0):
        self.fill_cnt = fill_cnt
        self.time = time
        self.df_raw_list = []

        file_list = glob.glob(input + "/*.xlsx")
        # print('[debug] len(file_list) = ', len(file_list))
        
        # file
        if len(file_list) == 0:
            df = pd.read_excel(input)
            fill_df = self.getFillDf(df)
            min_size = fill_df.columns.size

        # directory
        else:
            dn = pd.DataFrame(data={'line': ['0']}) # null padding
            df_list = []
            min_size = 999
            for f in file_list:
                df = pd.read_excel(f)
                self.df_raw_list.append(self.getFillDf(df))
                df_list.append(self.getFillDf(df))
                df_list.append(dn)
                if min_size > df.columns.size:
                    min_size = df.columns.size
            fill_df = pd.concat(df_list).drop('line', axis=1)

        # get all column name list (except 0, 1)
        target_all_list = [n for n in range(2, min_size)]
        # print('[debug] target_all_list = ', target_all_list)
        # print('[debug] min_size = ', min_size)
             
        if target_all:
            self.target = target_all_list
            self.target_name = fill_df.columns.tolist()[2:min_size]
        else:
            self.target = target
            self.target_name = []
            for t in target:
                self.target_name.append(fill_df.columns.tolist()[t])

        # print('[debug] self.target = ', self.target)

        self.group_cnt = self.time * len(self.target)
        # print('[debug] self.group_cnt (y) = ', self.group_cnt)

        self.target_df = self.getTargetDf(fill_df)
        # print('[debug] self.target_df = ', self.target_df)

    def getFillDf(self, df):
        if self.fill_cnt != 0:
            mask = df.copy()
            for i in df.columns: 
                dfx = pd.DataFrame( df[i] )
                dfx['new'] = ((dfx.notnull() != dfx.shift().notnull()).cumsum())
                dfx['ones'] = 1
                mask[i] = (dfx.groupby('new')['ones'].transform('count') < self.fill_cnt + 1) | df[i].notnull()
            df = df.interpolate().bfill()[mask]
        return df 

    def getTargetDf(self, df):
        return df.iloc[:, self.target]

    def getRelTargetDf(self, df):
        return df.loc[:, self.target_name]

    def getLabelDf(self, target_df):
        x = target_df.isna().any(axis='columns').astype(int)
        y = target_df.isna().any(axis='columns').astype(int).shift(periods=1, fill_value=0)
        z = pd.concat([x, y], axis=1).any(axis='columns').astype(int).cumsum()
        raw_label_df = pd.concat([x, z], axis=1)
        raw_label_df.columns = ['is_nan', 'group']
        new_df = pd.concat([target_df, raw_label_df], axis=1)
        label_df = new_df.where(new_df['is_nan'] != 1).dropna()
        return label_df

    def getSizeSr(self, label_df):
        size_sr = label_df.groupby(label_df['group']).size()
        return size_sr

    def getDiscard(self, target_df):
        return ( len(target_df) * len(self.target) ) % ( self.group_cnt )

    def sliceDf(self, target_df, discard):
        return target_df[:len(target_df) - int(discard / len(self.target))]

    def shiftDf(self, label_df, size_sr):
        target_idx_list = size_sr.where(size_sr >= self.time).dropna().index.tolist()
        target_df_list = []
        concat_list = []

        # debug
        # print('[debug] group_cnt = ', len(target_idx_list))
        
        for idx in target_idx_list:
            output_df = label_df.where(label_df['group'] == idx).dropna()
            total_cnt = len(label_df.where(label_df['group'] == idx).dropna())

            # debug
            # print('[debug] total_cnt = ', total_cnt)

            target_df_list.append({ 'total_cnt': total_cnt, 'output_df': output_df })
        for t in target_df_list:
            for n in range(0, t['total_cnt']-self.time+1):
                split_df = t['output_df'][n:self.time+n]
                concat_list.append(split_df)
                
        # validate
        if len(concat_list) == 0:
            # print('[fail] no groups were created. reduce the time option')
            exit(0)

        target_df = pd.concat(concat_list).drop('is_nan', axis=1).drop('group', axis=1)
        return target_df

    def getReshapeNp(self, output_np):
        output_np = output_np.reshape(-1, self.group_cnt)
        return output_np

    def getNp(self, output_df):
        output_np = output_df.to_numpy()
        return output_np

    # 삭제 예정
    def getDataSet(self):
        label_df = self.getLabelDf(self.target_df) # diff
        size_sr = self.getSizeSr(label_df) # diff
        target_df = self.shiftDf(label_df, size_sr) # diff
        discard = self.getDiscard(target_df)
        output_df = self.sliceDf(target_df, discard)
        output_np = self.getNp(output_df)
        return output_np

    # custom
    def getDataFrame(self):
        label_df = self.getLabelDf(self.target_df) # diff
        size_sr = self.getSizeSr(label_df) # diff
        target_df = self.shiftDf(label_df, size_sr) # diff
        discard = self.getDiscard(target_df)
        output_df = self.sliceDf(target_df, discard)
        return output_df

    def getRawDataSet(self):
        # file
        if len(self.df_raw_list) == 0:
            target_df = self.target_df
            discard = self.getDiscard(target_df)
            output_df = self.sliceDf(target_df, discard)
            output_np = self.getNp(output_df)
            return output_np

        # directory
        else:
            output_np_list = []
            for fill_df in self.df_raw_list:
                target_df = self.getRelTargetDf(fill_df)
                discard = self.getDiscard(target_df)
                output_df = self.sliceDf(target_df, discard)
                output_np = self.getNp(output_df)
                output_np_list.append(output_np)
            return output_np_list

    def npToExcel(self, input_np, save_path, timeFormat=False):
        df = pd.DataFrame(data=input_np)

        if timeFormat:
            hour_add = datetime.timedelta(hours = 1)
            target = datetime.datetime(2020, 1, 1, 0, 0) - hour_add
            date_list = []
            for day in range(0, len(df)):
                target = (target + hour_add)
                date_list.append(target.strftime("%m.%d %H:%S"))
            df.insert(loc=0, column='date', value=date_list)

        df.to_excel(save_path, index=False)

    def reverseReShape(self, input_np):
        return input_np.reshape(-1, len(self.target))

In [7]:
input = './data/가평_full_2019.xlsx'
preprocess = PreProcess(
    input=input, # file or directory
    target=[2,3,4,5,6,7,8,9,10], # if target_all is True, this option ignore
    time=3,
    target_all=False,
    fill_cnt=0
)
output_df = preprocess.getDataFrame()
output_np = output_df.to_numpy()
output_np

array([[2.800e+00, 6.800e+00, 9.700e+01, ..., 1.771e+00, 4.000e-03,
        3.800e+00],
       [2.800e+00, 6.800e+00, 9.700e+01, ..., 1.812e+00, 5.000e-03,
        4.100e+00],
       [2.800e+00, 6.800e+00, 9.700e+01, ..., 1.765e+00, 3.000e-03,
        3.800e+00],
       ...,
       [3.400e+00, 7.100e+00, 1.150e+02, ..., 1.738e+00, 2.000e-03,
        2.130e+01],
       [3.400e+00, 7.100e+00, 1.150e+02, ..., 1.723e+00, 2.000e-03,
        2.150e+01],
       [3.300e+00, 7.100e+00, 1.150e+02, ..., 1.705e+00, 2.000e-03,
        2.090e+01]])

In [18]:
data = output_np
_, dim = data.shape
norm_data = data.copy()
min_val = np.zeros(dim)
max_val = np.zeros(dim)

print('dim = ', dim)
print('norm_data = ', norm_data)
print('min_val = ', min_val)
print('max_val = ', max_val)

for i in range(dim):
    min_val[i] = np.nanmin(norm_data[:,i])
    norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
    max_val[i] = np.nanmax(norm_data[:,i])
    norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)
    
print('norm_data = ', norm_data)
print('min_val = ', min_val)
print('max_val = ', max_val)

dim =  9
norm_data =  [[2.800e+00 6.800e+00 9.700e+01 ... 1.771e+00 4.000e-03 3.800e+00]
 [2.800e+00 6.800e+00 9.700e+01 ... 1.812e+00 5.000e-03 4.100e+00]
 [2.800e+00 6.800e+00 9.700e+01 ... 1.765e+00 3.000e-03 3.800e+00]
 ...
 [3.400e+00 7.100e+00 1.150e+02 ... 1.738e+00 2.000e-03 2.130e+01]
 [3.400e+00 7.100e+00 1.150e+02 ... 1.723e+00 2.000e-03 2.150e+01]
 [3.300e+00 7.100e+00 1.150e+02 ... 1.705e+00 2.000e-03 2.090e+01]]
min_val =  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
max_val =  [0. 0. 0. 0. 0. 0. 0. 0. 0.]
norm_data =  [[0.0406504  0.41666632 0.25263158 ... 0.51838849 0.14814266 0.10357142]
 [0.0406504  0.41666632 0.25263158 ... 0.54232309 0.18517833 0.11428571]
 [0.0406504  0.41666632 0.25263158 ... 0.51488586 0.111107   0.10357142]
 ...
 [0.06504065 0.66666611 0.44210526 ... 0.49912405 0.07407133 0.7285714 ]
 [0.06504065 0.66666611 0.44210526 ... 0.49036749 0.07407133 0.73571426]
 [0.06097561 0.66666611 0.44210526 ... 0.47985961 0.07407133 0.71428569]]
min_val =  [ 1.8    6.3   73.    

In [9]:
import numpy as np
def normalization (data, parameters=None):
    '''Normalize data in [0, 1] range.

    Args:
        - data: original data

    Returns:
        - norm_data: normalized data
        - norm_parameters: min_val, max_val for each feature for renormalization
    '''

    # Parameters
    _, dim = data.shape
    norm_data = data.copy()

    if parameters is None:

        # MixMax normalization
        min_val = np.zeros(dim)
        max_val = np.zeros(dim)

        # For each dimension
        for i in range(dim):
            min_val[i] = np.nanmin(norm_data[:,i])
            norm_data[:,i] = norm_data[:,i] - np.nanmin(norm_data[:,i])
            max_val[i] = np.nanmax(norm_data[:,i])
            norm_data[:,i] = norm_data[:,i] / (np.nanmax(norm_data[:,i]) + 1e-6)

        # Return norm_parameters for renormalization
        norm_parameters = {'min_val': min_val,
                        'max_val': max_val}

    else:
        min_val = parameters['min_val']
        max_val = parameters['max_val']

        # For each dimension
        for i in range(dim):
            norm_data[:,i] = norm_data[:,i] - min_val[i]
            norm_data[:,i] = norm_data[:,i] / (max_val[i] + 1e-6)

        norm_parameters = parameters

    return norm_data, norm_parameters

def renormalization (norm_data, norm_parameters):
    '''Renormalize data from [0, 1] range to the original range.

    Args:
        - norm_data: normalized data
        - norm_parameters: min_val, max_val for each feature for renormalization

    Returns:
        - renorm_data: renormalized original data
    '''

    min_val = norm_parameters['min_val']
    max_val = norm_parameters['max_val']

    _, dim = norm_data.shape
    renorm_data = norm_data.copy()

    for i in range(dim):
        renorm_data[:,i] = renorm_data[:,i] * (max_val[i] + 1e-6)
        renorm_data[:,i] = renorm_data[:,i] + min_val[i]

    return renorm_data

In [10]:
normalization(output_np)

(array([[0.0406504 , 0.41666632, 0.25263158, ..., 0.51838849, 0.14814266,
         0.10357142],
        [0.0406504 , 0.41666632, 0.25263158, ..., 0.54232309, 0.18517833,
         0.11428571],
        [0.0406504 , 0.41666632, 0.25263158, ..., 0.51488586, 0.111107  ,
         0.10357142],
        ...,
        [0.06504065, 0.66666611, 0.44210526, ..., 0.49912405, 0.07407133,
         0.7285714 ],
        [0.06504065, 0.66666611, 0.44210526, ..., 0.49036749, 0.07407133,
         0.73571426],
        [0.06097561, 0.66666611, 0.44210526, ..., 0.47985961, 0.07407133,
         0.71428569]]),
 {'min_val': array([ 1.8  ,  6.3  , 73.   ,  6.2  ,  0.3  ,  1.1  ,  0.883,  0.   ,
          0.9  ]),
  'max_val': array([2.460e+01, 1.200e+00, 9.500e+01, 8.700e+00, 8.330e+01, 2.000e+00,
         1.713e+00, 2.700e-02, 2.800e+01])})